In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.5,
    height_shift_range=0.5,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True, verbose=1
)

# 构建改进的CNN模型
# 卷积神经网络
model = Sequential(
    [
        Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1), padding="same"),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),

        Flatten(),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(10, activation="softmax"),
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=200,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_optimized_v10.keras")

2025-01-11 14:09:21.245950: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-01-11 14:09:21.245976: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-11 14:09:21.245983: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-11 14:09:21.246017: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-11 14:09:21.246036: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


2025-01-11 14:09:22.493795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


936/937 [============================>.] - ETA: 0s - loss: 2.0412 - accuracy: 0.3101

2025-01-11 14:09:48.883700: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


937/937 [==============================] - 29s 26ms/step - loss: 2.0404 - accuracy: 0.3104 - val_loss: 0.7140 - val_accuracy: 0.7516 - lr: 0.0010
Epoch 2/200
937/937 [==============================] - 23s 25ms/step - loss: 1.2115 - accuracy: 0.5919 - val_loss: 0.2252 - val_accuracy: 0.9283 - lr: 0.0010
Epoch 3/200
937/937 [==============================] - 23s 25ms/step - loss: 0.9663 - accuracy: 0.6736 - val_loss: 0.1810 - val_accuracy: 0.9466 - lr: 0.0010
Epoch 4/200
937/937 [==============================] - 23s 25ms/step - loss: 0.8719 - accuracy: 0.7048 - val_loss: 0.1626 - val_accuracy: 0.9498 - lr: 0.0010
Epoch 5/200
937/937 [==============================] - 23s 25ms/step - loss: 0.8218 - accuracy: 0.7239 - val_loss: 0.1525 - val_accuracy: 0.9563 - lr: 0.0010
Epoch 6/200
937/937 [==============================] - 23s 24ms/step - loss: 0.7774 - accuracy: 0.7367 - val_loss: 0.1325 - val_accuracy: 0.9589 - lr: 0.0010
Epoch 7/200
937/937 [==============================] - 23s 25ms/